# 2020 NIMS 산업수학 아카데미 민들레 튜토리얼 
## 개화한 민들레 꽃의 갯수를 세는 프로그램 구축
* 4~5월 한 달간의 민들레 꽃을 찍은 데이터를 제공받음 (dandelion_640.zip)
* 데이터의 갯수가 너무 많으므로 꽃이 가장 많이 피어있는 4월 17일을 기준으로 앞으로 150장은 A팀이 뒤로 150장은 B팀이 라벨링 하기로 결정
    * http://www.robots.ox.ac.uk/~vgg/software/via/via.html 
    * 이 사이트에서 라벨링을 진행하였고 bounding box를 기본적인 직사각형으로 하기로 결정
    * 우리는 40, 40 35, 35을 나눠서 진행, B팀은 18, 19, 20, 21일을 36장씩 진행하기로 하였으나 20일 데이터에 이상이 있어 우리 데이터 75장과 18, 20일 데이터를 test셋 나머지를 triain셋으로 두고 작업을 진행
* 여러가지 모델을 두고 작업하다가 Detectron2의 faster_rcnn, retinanet 두 가지 모델로 결정

# 데이터 처리 환경 설정

In [ ]:
!pip install pyyaml==5.1
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())
assert torch.__version__.startswith("1.7")
!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.7/index.html
!gcc --version

exit(0)

In [ ]:
# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import os, json, cv2, random
from google.colab.patches import cv2_imshow

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

## 데이터 불러오기 train set과 test set으로 구분 

In [ ]:
!unzip ./drive/"My Drive"/dandelion.zip > /dev/null

* 이미지를 띄우기 위한 작업

In [ ]:
from detectron2.structures import BoxMode

def get_point_dicts(img_dir):
    json_file = os.path.join(img_dir, "via_region_data.json") # json파일 읽어오기
    with open(json_file) as f:
        imgs_anns = json.load(f)

    filename_list=[] #우리 json 파일은 dictionary안에 list가 있어 dictionary형태로 바꿔주는 작업
    for idx, v in enumerate(imgs_anns.values()):
        filename_list.append(v['filename']+str(v['size']))
    for j in range(len(filename_list)):
        imgs_anns[filename_list[j]]['regions']={str(i) : string for i , string in enumerate(imgs_anns[filename_list[j]]['regions'])}

    dataset_dicts = []
    for idx, v in enumerate(imgs_anns.values()):
        record = {}

        filename = os.path.join(img_dir, v["filename"])
        height, width = cv2.imread(filename).shape[:2]
        
        record["file_name"] = filename
        record["image_id"] = idx
        record["height"] = height
        record["width"] = width

        annos = v["regions"]
        
        objs = [] # 왼쪽 상단이 (0,0)이라 (x,y)에서 (w,h)를 더한게 우측하단, 그렇게 bbox 구성
        for _, anno in annos.items():
            assert not anno["region_attributes"]
            anno=anno['shape_attributes']
            px=float(anno['x'])
            py=float(anno['y'])
            pw=float(anno['width'])
            ph=float(anno['height'])
            poly=[(px+0.5*pw,py+0.5*ph)]
            poly = [p for x in poly for p in x]

            obj = {
                "bbox": [px, py, px+pw, py+ph],
                "bbox_mode": BoxMode.XYXY_ABS,
                # "segmentation": [poly],
                "category_id": 0,
            }
            objs.append(obj)
        record["annotations"] = objs
        dataset_dicts.append(record)
    return dataset_dicts

for d in ["train", "val"]: # catalog에 등록
    DatasetCatalog.register("dandelion_"+d, lambda d=d: get_point_dicts(d))
    MetadataCatalog.get("dandelion_"+d).set(thing_classes=["dandelion"])
dandelion_metadata = MetadataCatalog.get("dandelion_train")

* train set 이미지 확인

In [ ]:
dataset_dicts = get_point_dicts("train")
for d in random.sample(dataset_dicts, 1):
    img = cv2.imread(d["file_name"])
    visualizer = Visualizer(img[:, :, ::-1], metadata=dandelion_metadata, scale=0.5)
    out = visualizer.draw_dataset_dict(d)
    cv2_imshow(out.get_image()[:, :, ::-1])

# 훈련 부분
* faster_rcnn에서 R_101_FPN_3x가 50보다 좋은 성능을 보였기에 선택 
    * LR=0.0025, Max_Iter=500, BATCH_SIZE=512를 저장해두겠음_01
    * LR=0.0025, Max_Iter=500, BATCH_SIZE=128를 저장해두겠음_02
        * BATCH_SIZE를 줄이는게 효과가 좋다고 판단
    * LR=0.0025, Max_Iter=300, BATCH_SIZE=128를 저장해두겠음_03


In [ ]:
from detectron2.engine import DefaultTrainer

cfg = get_cfg() #config작업, parameter조정
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_101_FPN_3x.yaml")) # detectron2/coc-detection에서 모델 불러오기(github)
cfg.DATASETS.TRAIN = ("dandelion_train",)
cfg.DATASETS.TEST = ()
cfg.DATALOADER.NUM_WORKERS = 6 # 작업 대수
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_R_101_FPN_3x.yaml")  # Let training initialize from model zoo
cfg.SOLVER.IMS_PER_BATCH = 6
cfg.SOLVER.BASE_LR = 0.0025  # pick a good LR
cfg.SOLVER.MAX_ITER = 300    # 300 iterations seems good enough for this toy dataset; you will need to train longer for a practical dataset
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128   # faster, and good enough for this toy dataset (default: 512)
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1  # only has one class (ballon). (see https://detectron2.readthedocs.io/tutorials/datasets.html#update-the-config-for-new-datasets)

cfg.OUTPUT_DIR='drive/My Drive/models/faster_rcnn_R_101_FPN_3x_03' # 이 곳에 학습 결과를 저장해둠, 할 때마다 계속 학습하지 않고 학습 데이터를 불러와서 사용하기 위해
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)

trainer = DefaultTrainer(cfg) 
trainer.resume_or_load(resume=False)
trainer.train()


# retinanet 진행코드
# from detectron2.engine import DefaultTrainer

# cfg = get_cfg()
# cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/retinanet_R_101_FPN_3x.yaml"))
# cfg.DATASETS.TRAIN = ("dandelion_train",)
# cfg.DATASETS.TEST = ()
# cfg.DATALOADER.NUM_WORKERS = 6
# cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/retinanet_R_101_FPN_3x.yaml")  # Let training initialize from model zoo
# cfg.SOLVER.IMS_PER_BATCH = 6
# cfg.SOLVER.BASE_LR = 0.005  # pick a good LR
# cfg.SOLVER.MAX_ITER = 500   # 300 iterations seems good enough for this toy dataset; you will need to train longer for a practical dataset
# cfg.MODEL.RETINANET.BATCH_SIZE_PER_IMAGE = 128   # faster, and good enough for this toy dataset (default: 512)
# cfg.MODEL.RETINANET.NUM_CLASSES = 1  # retinanet은 faster_rcnn과 달라서 model다음에 쓰는게 다름, 이런 거는 detectron2 retinanet number of classes로 들어가서 documents를 확인해보면 사용 방법을 알 수 있음

# cfg.OUTPUT_DIR='drive/My Drive/models/retinanet_R_101_FPN_3x'
# os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)

# trainer = DefaultTrainer(cfg) 
# trainer.resume_or_load(resume=False)
# trainer.train()

* 훈련 결과 시각화

In [ ]:
# Look at training curves in tensorboard:
%load_ext tensorboard
%tensorboard --logdir output

# 예측 모델 생성
* 'drive/My Drive/models/faster_rcnn_R_101_FPN_3x' 여기 학습 결과가 저장되어 있기에 불러와서 예측 모델을 생성을 함

In [ ]:
cfg = get_cfg()

cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_101_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("dandelion_train",)
cfg.DATASETS.TEST = ()
cfg.DATALOADER.NUM_WORKERS = 20
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1  # only has one class (ballon)

# Inference should use the config with parameters that are used in training
# cfg now already contains everything we've set previously. We changed it a little bit for inference:
cfg.OUTPUT_DIR='drive/My Drive/models/faster_rcnn_R_101_FPN_3x_03'
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")  # path to the model we just trained
# 이유
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.85   # set a custom testing threshold
predictor = DefaultPredictor(cfg)

# retinanet 진행코드
# cfg = get_cfg()

# cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/retinanet_R_101_FPN_3x.yaml"))
# cfg.DATASETS.TRAIN = ("dandelion_train",)
# cfg.DATASETS.TEST = ()
# cfg.DATALOADER.NUM_WORKERS = 20
# cfg.MODEL.RETINANET.NUM_CLASSES = 1  # only has one class (ballon)

# # Inference should use the config with parameters that are used in training
# # cfg now already contains everything we've set previously. We changed it a little bit for inference:
# cfg.OUTPUT_DIR='drive/My Drive/models/retinanet_R_101_FPN_3x_01'
# cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")  # path to the model we just trained

# cfg.MODEL.RETINANET.SCORE_THRESH_TEST = 0.85   # set a custom testing threshold
# predictor = DefaultPredictor(cfg)

* Test dataset에 대해 잘 학습 되었는지 확인

In [ ]:
from detectron2.utils.visualizer import ColorMode

dataset_dicts = get_point_dicts("val")
for d in random.sample(dataset_dicts, 5):    
    im = cv2.imread(d["file_name"])
    outputs = predictor(im)
    v = Visualizer(im[:, :, ::-1], 
                   metadata=dandelion_metadata, 
                   scale=2, 
                   instance_mode=ColorMode.IMAGE_BW   # remove the colors of unsegmented pixels. This option is only available for segmentation models
    )
    out = v.draw_instance_predictions(outputs["instances"].to('cpu'))
    cv2_imshow(out.get_image()[:, :, ::-1])

* R2_score를 위해 리스트 생성

In [ ]:
real_ans=[]
predict_ans=[]
for d in dataset_dicts:
    im = cv2.imread(d["file_name"])
    outputs = predictor(im)
    real_ans.append(len(d['annotations']))
    predict_ans.append(len(outputs['instances'].to('cpu').scores))

In [ ]:
# evaluation 작업
# !rm -rf output

## Evaluation 검증
* AP50

In [ ]:
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader

evaluator = COCOEvaluator("dandelion_val", ("bbox",), False, output_dir="./output/", )
val_loader = build_detection_test_loader(cfg, "dandelion_val")
print(inference_on_dataset(predictor.model, val_loader, evaluator))
# another equivalent way to evaluate the model is to use `trainer.test`

* R2_score

In [ ]:
from sklearn.metrics import r2_score
r2_score(real_ans, predict_ans)

* Difference

In [ ]:
import pandas as pd
# [i for i in range(len(real_ans))]
re=pd.Series(real_ans)
pre=pd.Series(predict_ans)
err=pd.concat([re, pre], axis=1)
err.columns=['실제','예측']
err['오차']=err['실제']-err['예측']
err

sum(abs(err['오차'])), sum(err['실제'])

In [ ]:
err